In [2]:
import math
import numpy as np
from scqbf.scqbf_instance import *
from scqbf.scqbf_evaluator import *
import random
import time

In [3]:
instance = read_max_sc_qbf_instance("instances/sample_instances/1.txt")
instance

ScQbfInstance(n=5, subsets=[{1, 2}, {2, 3, 4}, {1, 4}, {3, 5}, {4, 5}], A=[[3.0, 1.0, -2.0, 0.0, 3.0], [0.0, -1.0, 2.0, 1.0, -1.0], [0.0, 0.0, 2.0, -2.0, 4.0], [0.0, 0.0, 0.0, 0.0, 5.0], [0.0, 0.0, 0.0, 0.0, 3.0]])

In [4]:
eval = ScQbfEvaluator(instance)

In [5]:
sample_solution = ScQbfSolution([1, 2])
eval.evaluate_coverage(sample_solution)

0.8

In [ ]:
class ScQbfGrasp:
    
    def __init__(self, instance: ScQbfInstance, iterations,
                 config: dict = {
                    "construction_method": "traditional",  # traditional | random_plus_greedy | sampled_greedy
                    "construction_args": (),
                    "local_search_method": "best_improve"  # best_improve | first_improve
                 }, time_limit_secs: float = None):
        self.instance = instance
        self.iterations = iterations
        self.config = config
        self.time_limit_secs = time_limit_secs
        self.solve_time = 0
        self.evaluator = ScQbfEvaluator(instance)


    def solve(self) -> ScQbfSolution:
        if self.instance is None:
            raise ValueError("Problem instance is not initialized")
        
        best_sol = ScQbfSolution([])
        start_time = time.perf_counter()
        for i in range(self.iterations):
            constructed_sol = self._constructive_heuristic()
            print(f"Constructed solution (iteration {i}): {constructed_sol.elements}")
            
            if not self.evaluator.is_solution_valid(constructed_sol):
                print("Constructed solution is not feasible, fixing...")
                constructed_sol = self._fix_solution(constructed_sol)
            
            sol = self._local_search(constructed_sol)
            
            if (self.evaluator.evaluate_objfun(sol) > self.evaluator.evaluate_objfun(best_sol)):
                best_sol = sol
            
            self.solve_time = time.perf_counter() - start_time
            if self.time_limit_secs is not None and self.solve_time >= self.time_limit_secs:
                break
            
        return best_sol
    
    def _fix_solution(self, sol: ScQbfSolution) -> ScQbfSolution:
        """
        This function is called when the constructed solution is not feasible.
        It'll add the most covering elements until the solution is feasible.
        """
        while not self.evaluator.is_solution_valid(sol):
            cl = [i for i in range(self.instance.n) if i not in sol.elements]
            best_cand = None
            best_coverage = -1
            
            for cand in cl:
                coverage = self.evaluator.evaluate_insertion_delta_coverage(cand, sol)
                if coverage > best_coverage:
                    best_coverage = coverage
                    best_cand = cand
            
            if best_cand is not None:
                sol.elements.append(best_cand)
            else:
                break
        
        if not self.evaluator.is_solution_valid(sol):
            raise ValueError("Could not fix the solution to be feasible")
        
        return sol

    def _constructive_heuristic(self) -> ScQbfSolution:
        if self.config["construction_method"] == "traditional":
            alpha = self.config["construction_args"][0] if len(self.config.get("construction_args", [])) > 0 else 0.5
            return self._constructive_heuristic_traditional(alpha)

        elif self.config["construction_method"] == "random_plus_greedy":
            alpha, p = self.config["construction_args"] if len(self.config.get("construction_args", [])) > 0 else (0.5, 0.5)
            return self._constructive_heuristic_random_plus_greedy(alpha, p)
        
        elif self.config["construction_method"] == "sampled_greedy":
            p = self.config["construction_args"][0] if len(self.config.get("construction_args", [])) > 0 else 0.1
            return self._constructive_heuristic_sampled_greedy(p)
        
        else:
            return self._constructive_heuristic_traditional()

    def _constructive_heuristic_traditional(self, alpha: float) -> ScQbfSolution:
        constructed_sol = ScQbfSolution([])
        cl = [i for i in range(self.instance.n)] # makeCl

        prev_objfun = float("-inf")
        while(prev_objfun < self.evaluator.evaluate_objfun(constructed_sol)): # Constructive Stop Criteria
            # traditional constructive heuristic
            rcl = []
            min_delta = math.inf
            max_delta = -math.inf
            cl = [i for i in cl if i not in constructed_sol.elements] # update_cl
            
            prev_objfun = self.evaluator.evaluate_objfun(constructed_sol)
            
            for candidate_element in cl:
                delta_objfun = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                if delta_objfun < min_delta:
                    min_delta = delta_objfun
                if delta_objfun > max_delta:
                    max_delta = delta_objfun
            
            # This is where we define the RCL.
            for candidate_element in cl:
                delta_objfun = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                if delta_objfun >= (min_delta + alpha * (max_delta - min_delta)):

                    ## ONLY add to rcl if coverage increases
                    if self.evaluator.evaluate_insertion_delta_coverage(candidate_element, constructed_sol) > 0:
                        rcl.append(candidate_element)

            # Randomly select an element from the RCL to add to the solution
            if rcl:
                chosen_element = random.choice(rcl)
                constructed_sol.elements.append(chosen_element)

        return constructed_sol

    def _constructive_heuristic_random_plus_greedy(self, alpha: float, p: float):
        if p < 0:
            p = 0
        if p > 1:
            p = 1
        constructed_sol = ScQbfSolution([])
        cl = [i for i in range(self.instance.n)] # makeCl

        prev_objfun = float("-inf")
        target_cover = math.floor(p * self.instance.n)
        while self.evaluator.evaluate_coverage(self.constructed_sol) < target_cover:
        
            rcl = []
            min_delta = math.inf
            max_delta = -math.inf
            cl = [i for i in cl if i not in constructed_sol.elements] # update_cl
            
            
            prev_objfun = self.evaluator.evaluate_objfun(constructed_sol)
            
            for candidate_element in cl:
                delta_objfun = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                if delta_objfun < min_delta:
                    min_delta = delta_objfun
                if delta_objfun > max_delta:
                    max_delta = delta_objfun
            
            # This is where we define the RCL.
            for candidate_element in cl:
                delta_objfun = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                if delta_objfun >= (min_delta + alpha * (max_delta - min_delta)):

                    ## ONLY add to rcl if coverage increases
                    if self.evaluator.evaluate_insertion_delta_coverage(candidate_element, constructed_sol) > 0:
                        rcl.append(candidate_element)

            # Randomly select an element from the RCL to add to the solution
            if rcl:
                chosen_element = random.choice(rcl)
                constructed_sol.elements.append(chosen_element)
            
            if not (prev_objfun < self.evaluator.evaluate_objfun(constructed_sol)):
                break

        prev_objfun = float("-inf")

        while(prev_objfun < self.evaluator.evaluate_objfun(constructed_sol)): # Constructive Stop Criteria
            cl = [i for i in cl if i not in constructed_sol.elements] # update_cl
            
            
            prev_objfun = self.evaluator.evaluate_objfun(constructed_sol)
            best_delta = float("-inf")
            best_cand_in = -1
            # This is where we define the RCL.
            for candidate_element in cl:
                delta_objfun = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                ## ONLY add to rcl if coverage increases
                if self.evaluator.evaluate_insertion_delta_coverage(candidate_element, constructed_sol) > 0 \
                and delta_objfun > best_delta \
                and prev_objfun < self.evaluator.evaluate_objfun(constructed_sol):
                    best_cand_in = candidate_element
                    best_delta = delta_objfun
            
            if (best_cand_in >= 0):
                constructed_sol.elements.append(best_cand_in)

        return constructed_sol
    
    def _constructive_heuristic_sampled_greedy(self, p: int):
        constructed_sol = ScQbfSolution([])
        cl = [i for i in range(self.instance.n)] # makeCl

        prev_objfun = float("-inf")
        while(prev_objfun < self.evaluator.evaluate_objfun(constructed_sol)): # Constructive Stop Criteria
            cl = [i for i in cl if i not in constructed_sol.elements] # update_cl
            prev_objfun = self.evaluator.evaluate_objfun(constructed_sol)
            
            rcl = random.sample(cl, min(len(cl), p))
            best_delta = float("-inf")
            best_cand_in = None
            for candidate_element in rcl:
                delta = self.evaluator.evaluate_insertion_delta(candidate_element, constructed_sol)
                if delta > best_delta:
                    best_delta = delta
                    best_cand_in = candidate_element
            
            if best_delta > 0 and best_cand_in is not None:
                constructed_sol.elements.append(best_cand_in)
            else:
                break
        
        return constructed_sol

    ####################

    def _local_search(self, starting_point: ScQbfSolution) -> ScQbfSolution:
        if self.config.get("local_search_method", False) == "best_improve":
            return self._local_search_best_improve(starting_point)
        elif self.config.get("local_search_method", False) == "first_improve":
            return self._local_search_first_improve(starting_point)


    def _local_search_best_improve(self, starting_point: ScQbfSolution) -> ScQbfSolution:
        sol = ScQbfSolution(starting_point.elements.copy())
        
        _search_iterations = 0
        
        while True:
            _search_iterations += 1
            
            best_delta = float("-inf")
            best_cand_in = None
            best_cand_out = None

            cl = [i for i in range(self.instance.n) if i not in sol.elements]

            # Evaluate insertions
            for cand_in in cl:
                delta = self.evaluator.evaluate_insertion_delta(cand_in, sol)
                if delta > best_delta:
                    best_delta = delta
                    best_cand_in = cand_in
                    best_cand_out = None

            # Evaluate removals
            for cand_out in sol.elements:
                delta = self.evaluator.evaluate_removal_delta(cand_out, sol)
                if delta > best_delta:
                    # Check if removing this element would break feasibility
                    temp_sol = ScQbfSolution(sol.elements.copy())
                    temp_sol.elements.remove(cand_out)
                    if self.evaluator.is_solution_valid(temp_sol):
                        best_delta = delta
                        best_cand_in = None
                        best_cand_out = cand_out

            # Evaluate exchanges
            for cand_in in cl:
                for cand_out in sol.elements:
                    delta = self.evaluator.evaluate_exchange_delta(cand_in, cand_out, sol)
                    if delta > best_delta:
                        # Check if this exchange would break feasibility
                        temp_sol = ScQbfSolution(sol.elements.copy())
                        temp_sol.elements.remove(cand_out)
                        temp_sol.elements.append(cand_in)
                        if self.evaluator.is_solution_valid(temp_sol):
                            best_delta = delta
                            best_cand_in = cand_in
                            best_cand_out = cand_out

            # Apply the best move if it improves the solution
            if best_delta > 0:  # Positive delta means improvement for maximization
                print(f"[local_search]: Improvement found! Delta: {best_delta}, in {best_cand_in}, out {best_cand_out}")
                if best_cand_in is not None:
                    sol.elements.append(best_cand_in)
                if best_cand_out is not None:
                    sol.elements.remove(best_cand_out)

                self.evaluator.evaluate_objfun(sol)
            else:
                print(f"[local_search]: No improvement found after ({_search_iterations}) iterations!")
                break  # No improving move found
        
        return sol

    def _local_search_first_improve(self, starting_point: ScQbfSolution) -> ScQbfSolution:
        sol = ScQbfSolution(starting_point.elements.copy())
        _search_iterations = 0
        
        while True:
            _search_iterations += 1
            improvement_found = False
            
            cl = [i for i in range(self.instance.n) if i not in sol.elements]

            # Insertions
            for cand_in in cl:
                delta = self.evaluator.evaluate_insertion_delta(cand_in, sol)
                if delta > 0:
                    print(f"[local_search]: First improvement found (insertion)! Delta: {delta}, in {cand_in}")
                    sol.elements.append(cand_in)
                    self.evaluator.evaluate_objfun(sol)
                    improvement_found = True
                    break
            
            if improvement_found:
                continue
            
            # Removals
            for cand_out in sol.elements:
                delta = self.evaluator.evaluate_removal_delta(cand_out, sol)
                if delta > 0: 
                    # Check if removing this element would break feasibility
                    temp_sol = ScQbfSolution(sol.elements.copy())
                    temp_sol.elements.remove(cand_out)
                    if self.evaluator.is_solution_valid(temp_sol):
                        print(f"[local_search]: First improvement found (removal)! Delta: {delta}, out {cand_out}")
                        sol.elements.remove(cand_out)
                        self.evaluator.evaluate_objfun(sol)
                        improvement_found = True
            
            if improvement_found:
                continue

            # Exchanges
            for cand_in in cl:
                for cand_out in sol.elements:
                    delta = self.evaluator.evaluate_exchange_delta(cand_in, cand_out, sol)
                    if delta > 0:
                        # Check if this exchange would break feasibility
                        temp_sol = ScQbfSolution(sol.elements.copy())
                        temp_sol.elements.remove(cand_out)
                        temp_sol.elements.append(cand_in)
                        if self.evaluator.is_solution_valid(temp_sol):
                            print(f"[local_search]: First improvement found (exchange)! Delta: {delta}, in {cand_in}, out {cand_out}")
                            sol.elements.remove(cand_out)
                            sol.elements.append(cand_in)
                            self.evaluator.evaluate_objfun(sol)
                            improvement_found = True
                            break
                
                if improvement_found:
                    break

            if improvement_found:
                continue
            print(f"[local_search]: No improvement found after ({_search_iterations}) iterations!")
            break
        
        return sol
        

In [7]:
# Test the ScQbfGrasp implementation
instance = read_max_sc_qbf_instance("instances/sample_instances/2.txt")
grasp = ScQbfGrasp(instance, iterations=10)

print("Testing GRASP algorithm...")
print(f"Instance size: n = {instance.n}")
print(f"Number of subsets: {len(instance.subsets)}")

# Run GRASP
best_solution = grasp.solve()

evaluator = ScQbfEvaluator(instance)
print(f"\nBest solution found:")
print(f"Selected elements: {best_solution.elements}")
print(f"Objective value: {evaluator.evaluate_objfun(best_solution)}")
print(f"Coverage: {evaluator.evaluate_coverage(best_solution):.2%}")
print()

# Test with different alpha values
print("\nTesting different alpha values:")
alphas = [0.0, 0.3, 0.7, 1.0]
for alpha in alphas:
    grasp_test = ScQbfGrasp(instance, iterations=5, config={
        "construction_method": "traditional",
        "construction_args": (alpha,),
        "local_search_method": "best_improve"
    })
    solution = grasp_test.solve()
    obj_val = evaluator.evaluate_objfun(solution)
    coverage = evaluator.evaluate_coverage(solution)
    print(f"Alpha {alpha}: Obj={obj_val:.2f}, Coverage={coverage:.2%}, Solution={solution.elements}")
    print()

Testing GRASP algorithm...
Instance size: n = 3
Number of subsets: 3
Constructed solution (iteration 0): [2, 1]
[local_search]: Improvement found! Delta: 2.0, in 0, out None
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 1): [1, 0]
[local_search]: Improvement found! Delta: 1.5, in 2, out None
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 2): [1, 0]
[local_search]: Improvement found! Delta: 1.5, in 2, out None
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 3): [1, 0]
[local_search]: Improvement found! Delta: 1.5, in 2, out None
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 4): [1, 0]
[local_search]: Improvement found! Delta: 1.5, in 2, out None
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 5): [2, 1]
[local_search]: Improvement found! Delta: 2.0, in 0, out None
[local

### Testing sampled greedy construction

In [8]:
instance = read_max_sc_qbf_instance("instances/sample_instances/2.txt")
grasp = ScQbfGrasp(instance, iterations=10, config={
    "construction_method": "sampled_greedy",
    "construction_args": (instance.n // 10,),
    "local_search_method": "first_improve"
}
)

best_solution = grasp.solve()

evaluator = ScQbfEvaluator(instance)
print(f"\nBest solution found:")
print(f"Selected elements: {best_solution.elements}")
print(f"Objective value: {evaluator.evaluate_objfun(best_solution)}")
print(f"Coverage: {evaluator.evaluate_coverage(best_solution):.2%}")
print()

Constructed solution (iteration 0): []
Constructed solution is not feasible, fixing...
[local_search]: First improvement found (insertion)! Delta: 1.5, in 2
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 1): []
Constructed solution is not feasible, fixing...
[local_search]: First improvement found (insertion)! Delta: 1.5, in 2
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 2): []
Constructed solution is not feasible, fixing...
[local_search]: First improvement found (insertion)! Delta: 1.5, in 2
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 3): []
Constructed solution is not feasible, fixing...
[local_search]: First improvement found (insertion)! Delta: 1.5, in 2
[local_search]: No improvement found after (2) iterations!
Constructed solution (iteration 4): []
Constructed solution is not feasible, fixing...
[local_search]: First improvement found (insertio

# Testing on larger instances


In [13]:
instance = read_max_sc_qbf_instance("instances/gen1/instance1.txt")
grasp = ScQbfGrasp(instance, iterations=10, config={
    "construction_method": "random_plus_greedy",
    "construction_args": (),
    "local_search_method": "first_improve"
},
)

best_solution = grasp.solve()

evaluator = ScQbfEvaluator(instance)
print(f"\nBest solution found:")
print(f"Selected elements: {best_solution.elements}")
print(f"Objective value: {evaluator.evaluate_objfun(best_solution)}")
print(f"Coverage: {evaluator.evaluate_coverage(best_solution):.2%}")
print(f"Elapsed time: {grasp.solve_time:.2f} seconds")
print()

Constructed solution (iteration 0): [35, 79, 177, 25, 164, 161]
[local_search]: First improvement found (insertion)! Delta: 7.560000000000001, in 0
[local_search]: First improvement found (insertion)! Delta: 0.5900000000000005, in 1
[local_search]: First improvement found (insertion)! Delta: 12.93, in 5
[local_search]: First improvement found (insertion)! Delta: 11.979999999999999, in 10
[local_search]: First improvement found (insertion)! Delta: 0.35999999999999943, in 7
[local_search]: First improvement found (insertion)! Delta: 26.990000000000002, in 11
[local_search]: First improvement found (insertion)! Delta: 13.61, in 13
[local_search]: First improvement found (insertion)! Delta: 3.0500000000000007, in 12
[local_search]: First improvement found (insertion)! Delta: 27.660000000000004, in 19
[local_search]: First improvement found (insertion)! Delta: 7.7700000000000005, in 23
[local_search]: First improvement found (insertion)! Delta: 12.09, in 26
[local_search]: First improvement